<a href="https://colab.research.google.com/github/GustavoFPS/Estatistica-com-Python/blob/main/TEFE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
def DerivativeSigmoide(t, a, b, c):
    '''
    Função para o cálculo das derivadas parciais para a função Sigmói
    '''
    denominador = (1+np.exp(-b*(t-c)))**2
    dsda = 1/np.sqrt(denominador)
    dsdb = (a*(t-c)*np.exp(-b*(t-c)))/denominador
    dsdc = -(a*b*np.exp(-b*(t-c)))/denominador
    return np.array([dsda, dsdb, dsdc]).T

def DerivativeGompertz(t, d, f, h):
    '''
    Função para o cálculo das derivadas parciais para a função de Gom
    '''
    dgdd = np.exp(-f*np.exp(-h*t))
    dgdf = -d*np.exp(-h*t)*np.exp(-f*np.exp(-h*t))
    dgdh = d*f*t*np.exp(-h*t)*np.exp(-f*np.exp(-h*t))
    return np.array([dgdd, dgdf, dgdh]).T
def H_Matrix(func_derivative, t, V, y_data, *params):
    '''
    Realiza do cálculo da matriz H
    Imput -> Função das derivadas parciais da função desejada
    variável independente -> t
    vetor das variancias experimentias - V
    *params -> valores dos parâmetros quais a função das der
    '''
    X = func_derivative(t, *params)
    V_inv = np.linalg.inv(np.diag(V))
    cov = np.linalg.inv(X.T @ V_inv @ X)
    H = X @ cov @ X.T@V_inv
    Var_Res = (np.diag(np.ones(shape = len(V))) - H)@np.diag(V)
    res = (np.diag(np.ones(shape = len(V))) - H)@y_data
    return H, cov, Var_Res, res

def latex(array_numpy, num_casas_decimais = 2):
  '''
  '''

  # Formata a matriz com o número de casas decimais especificado
  array_formatado = np.around(array_numpy, decimals=num_casas_decimais)

  # Converte a matriz para uma string latex
  latex_matrix = np.array2string(array_formatado, separator=' &',
                                 suppress_small=True,
                                 formatter={'float_kind': lambda x: "{:.{}f}".format(x, num_casas_decimais)}).replace('[', '').replace(']', '').replace('\n', '\\\\\n').replace(' ', '')
  latex_matrix = '\\begin{bmatrix}\n' + latex_matrix + '\n\\end{bmatrix}'

  return latex_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import os
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/dadosAtividadePratica3.dat", delimiter='\t')
print(data)

Estimando (a)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Definindo a função sigmoide
def sigmoid(t, a, b, c):
    return a / (1 + np.exp(-b * (t - c)))

# importa os dados e os chutes iniciais
t_data = data['t']
y_data = data[' y ']
s_y = data['sigma']

a = 18
b = 0.3
c = 12


# Calculando os valores da função sigmoide
y2_data = sigmoid(t_data, a, b, c)

# Plot
plt.scatter(t_data, y_data, label='Dados', color = 'r')
plt.plot(t_data, y2_data, label=f'Sigmoide(a={a}, b={b}, c={c})')
plt.xlabel('t')
plt.ylabel('s(t)')
plt.title('Função Sigmoide')
plt.legend()
plt.show()


Estimativa de a, b e c

In [ ]:
chute = [a, b, c]

# Fit
popt, V_y = curve_fit(sigmoid, t_data, y_data, chute, absolute_sigma = True, sigma = s_y, jac = DerivativeSigmoide)
a, b, c = popt[0], popt[1], popt[2]

params_sig = [a, b, c]

# Parâmetros ajustados
print(f"Parâmetros ajustados: a: {popt[0]} +- {V_y[0][0]**0.5}, b:{popt[1]}+- {V_y[1][1]**0.5} e c:{popt[2]}+- {V_y[2][2]**0.5}" )


# residuos
H, V_p, V_r, res = H_Matrix(DerivativeSigmoide, t_data, s_y, y_data, *params_sig)


norm_res = res / np.sqrt(np.diag(V_r))

S = res.T @ np.linalg.inv(np.diag(s_y)) @ res

# chi2
chi2_sig = np.sum(((y_data - sigmoid(t_data,  a,b, c))**2)/(s_y**2))
N_u = len(t_data) - len(params_sig) # graus de liberdade
print('chi^2:', chi2_sig, 'chi_red²:', np.round(chi2_sig/N_u, 2), 'S : ', S, ' N - u: ', N_u)


# Plot
fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, sharex = True,
                               figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})

ax1.scatter(t_data, y_data, label='Dados')
ax1.plot(t_data, sigmoid(t_data, *popt), color='red',label=f'Ajuste (a={np.round(a, 2)}, b={np.round(b, 3)}, c={np.round(c, 2)})')
ax1.legend()
ax1.errorbar(t_data, y_data, yerr=s_y,fmt="o", label="Dados Experimentais")
ax1.set_title("Sigmoide")
ax2.scatter(t_data, norm_res)
ax2.hlines(y=0, xmin = 0, xmax = max(t_data), color = "r", linestyle = "-.")
ax2.errorbar(t_data, norm_res, yerr= 1, fmt='none')
ax2.set_ylabel("Resíduo normalizado")
plt.xlabel("t (s)")
fig.subplots_adjust(hspace=0.02)
plt.show()



In [ ]:
print(latex(V_y, 5))

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Definindo a função de Gompertz
def gompertz(t, d, g, h):
    return d * np.exp(-g * np.exp(-h * t))

#importando os dados e os chutes iniciais
t_data = data['t']
y_data = data[' y ']
s_y = data['sigma']

d = 24
g = 7
h = 0.15


# Calculando os valores da função sigmoide
y2_data = gompertz(t_data, d, g, h)

# Plot
plt.scatter(t_data, y_data, label='Dados', color = 'r')
plt.plot(t_data, y2_data, label=f'Gompertz(d={d}, g={g}, h={h})')
plt.xlabel('t')
plt.ylabel('g(t)')
plt.title('Função de Gompertz')
plt.legend()
plt.show()



In [ ]:
chute = [d, g, h]

# Ajuste de curva
popt, V_yg = curve_fit(gompertz, t_data, y_data, chute, absolute_sigma = True, sigma = s_y, jac = DerivativeGompertz)
d, g, h = popt[0], popt[1], popt[2]

# Parâmetros
params = [d, g, h]
print(f"Parâmetros ajustados: d: {popt[0]}+- {V_yg[0][0]**0.5}, g:{popt[1]}+- {V_yg[1][1]**0.5} e h:{popt[2]}+- {V_yg[2][2]**0.5}" )

# Residuos
H, V_p, V_r, res = H_Matrix(DerivativeGompertz, t_data, s_y, y_data, *params)

norm_res = res / np.sqrt(np.diag(V_r))

S = res.T @ np.linalg.inv(np.diag(s_y)) @ res

# chi2
chi2_gom = np.sum(((y_data - gompertz(t_data,  d,g, h))**2)/(s_y**2))

print('chi^2:', chi2_gom, 'chi_red²: ', np.round(chi2_gom/N_u,2),  'S : ', S, ' N - u: ', N_u)

# plot
fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, sharex = True,
                               figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})


ax1.scatter(t_data, y_data, label='Dados')
ax1.plot(t_data, gompertz(t_data, *popt), label=f'Ajuste (d={np.round(popt[0], 2)}, g={np.round(popt[1], 2)}, h:{np.round(popt[2], 2)})', color='red')
ax1.legend()
ax1.errorbar(t_data, y_data, yerr=s_y,fmt="o", label="Dados Experimentais")

ax2.scatter(t_data, norm_res)
ax2.hlines(y=0, xmin = 0, xmax = max(t_data), color = "r", linestyle = "-.")
ax2.errorbar(t_data, norm_res, yerr= 1, fmt='none')
ax2.set_ylabel("Resíduo normalizado")
plt.xlabel("t (s)")
fig.subplots_adjust(hspace=0.02)
ax1.set_title('Gompertz')
plt.show()

In [ ]:
print(latex(V_yg, 6))

In [ ]:
# função soma quadrada dos resíduos
def Q(sigmoid, a, b, c, t_data, y_data, s_y, modo = 'ab'):
  if modo == 'ab':
    Z = np.zeros_like(a)
    for i in range(b.shape[0]):
      for j in range(b.shape[1]):
            Z[i, j] = np.sum(((y_data - sigmoid(t_data, a[i, j], b[i, j], c))**2)/(s_y**2))

  elif modo == 'cb':
    Z = np.zeros_like(b)
    for i in range(b.shape[0]):
      for j in range(b.shape[1]):
            Z[i, j] = np.sum(((y_data - sigmoid(t_data, a, b[i, j], c[i, j]))**2)/(s_y**2))

  elif modo == 'ac':
    Z = np.zeros_like(a)
    for i in range(a.shape[0]):
      for j in range(a.shape[1]):
            Z[i, j] = np.sum(((y_data - sigmoid(t_data,  a[i, j], b , c[i, j]))**2)/(s_y**2))
  return Z

a_values = np.linspace(17.5, 18.5, 100)
b_values = np.linspace(0.2, 0.4, 100)
c_values = np.linspace(10.5, 13.5, 100)

A_1, B_1 = np.meshgrid( a_values, b_values)
C_2, B_2 = np.meshgrid( c_values, b_values)
A_3, C_3 = np.meshgrid( a_values, c_values)

a, b, c = params_sig

In [ ]:
Z1 = Q(sigmoid, A_1, B_1, c, t_data, y_data, s_y, modo  = 'ab')

In [ ]:
Z2 = Q(sigmoid, a, B_2, C_2, t_data, y_data, s_y, modo  = 'cb')

In [ ]:
Z3 = Q(sigmoid, A_3, b, C_3, t_data, y_data, s_y, modo  = 'ac')

In [ ]:

from mpl_toolkits.axes_grid1 import make_axes_locatable

# Níveis
contour_levels = []
for n in range (0, 6):
  contour_levels.append(chi2_sig + n**2)


# Plot
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 1, ncols = 3, sharex = False,
                                    figsize=(20, 8))

ax1.contour(A_1, B_1, Z1, levels=contour_levels, colors='red')
ax1.contourf(A_1, B_1, Z1, levels=contour_levels, cmap='twilight')
ax1.set_ylim(0.28, 0.32)
ax1.set_xlim(17.6, 18.3)
ax1.set_xlabel('a')
ax1.set_ylabel('b')
ax1.grid(True)


ax2.contour(B_2, C_2, Z2, levels=contour_levels, colors='red')
ax2.contourf(B_2, C_2, Z2, levels=contour_levels, cmap='twilight')
ax2.set_ylim(11.75, 12.25)
ax2.set_xlim(0.28, 0.32)
ax2.set_xlabel('b')
ax2.set_ylabel('c')
ax2.grid(True)

ax3.contour(A_3, C_3, Z3, levels=contour_levels, colors='red')
contour = ax3.contourf(A_3, C_3, Z3, levels=contour_levels, cmap='twilight')
ax3.set_ylim(11.75, 12.25)
ax3.set_xlim(17.6, 18.3)
ax3.set_xlabel('a')
ax3.set_ylabel('c')
ax3.grid(True)

# Barra de cores
divider = make_axes_locatable(ax3)
cax = divider.append_axes("right", size="5%", pad=0.1)
cbar = plt.colorbar(contour, cax=cax)

# Titulos
fig.suptitle('Curvas de nível para Sigmoide', fontsize=16)
cbar.set_label('Chi²')


plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2

graus_de_liberdade = N_u  # Graus de liberdade (k)
valor_q = chi2_gom  # Valor para o qual você deseja calcular a CDF

cdf_valor = chi2.cdf(valor_q, graus_de_liberdade)
print(f"A CDF para ChiSquare({graus_de_liberdade}) com valor {valor_q:.2f} é {cdf_valor:.16f}")

In [ ]:
import scipy.stats as stats

#limites
nivel_significancia_inf = 0.01
nivel_significancia_sup = 0.95

graus_de_liberdade = N_u

chi_red = chi2_sig / N_u


# valores críticos
valor_critico_inf = stats.chi2.ppf(nivel_significancia_inf, graus_de_liberdade)
valor_critico_sup = stats.chi2.ppf(nivel_significancia_sup, graus_de_liberdade)

qui_quadrado_reduzido_minimo_inf = valor_critico_inf / graus_de_liberdade
qui_quadrado_reduzido_minimo_sup = valor_critico_sup / graus_de_liberdade

print(f"Qui-quadrado reduzido: {qui_quadrado_reduzido_minimo_inf:.4f} <= {chi_red:.4f} <= {qui_quadrado_reduzido_minimo_sup:.4f}")
